# Gradiente descendente Estocástico

In [1]:
import pandas as pd
import numpy as np
import math

import time

In [2]:
# Importando .csv
train = pd.read_csv('dataset/train_data.csv', header=None,  skiprows=[0], usecols=[0,1,2])

In [3]:
# Função FunkSVD
def funkSVD(R, k, lr = 0.05, reg = 0.02, miter = 10):
    global_mean = R[2].mean(skipna = True)
    nusers = df[0].max()
    nitems = df[1].max()
    bu = np.full(nusers+1, 0)
    bi = np.full(nusers+1, 0)
    P = np.full((nusers+1, k), 0.1)
    Q = np.full((nitems+1, k), 0.1)
    for f in range(k):
        for l in range(0, miter):
            for j in range(0, len(R)):
                u = R[0][j]
                i = R[1][j]
                r_ui = R[2][j]
                pred = global_mean + bu[u] + bi[i] + np.dot(P[u, ], Q[i, ])
                e_ui = r_ui - pred
                bu[u] = bu[u] + lr * e_ui # - reg * bu[u]
                bi[i] = bi[i] + lr * e_ui # - reg * bi[i]
                temp_uf = P[u, f]
                P[u, f] = P[u, f] + lr * (e_ui * Q[i, f] - reg * P[u, f])
                Q[i, f] = Q[i, f] + lr * (e_ui * temp_uf - reg * Q[i, f])

    return { "global_mean": global_mean, "bu": bu, "bi": bi, "P": P, "Q": Q }

In [4]:
# funk = funkSVD(train, 4)

In [5]:
# Função SVDOtimizado
def svdopt(train, k, lr = 0.05, reg = 0.02, miter = 10):
    global_mean = train[2].mean(skipna = True)
    nusers = train[0].max()+1
    nitems = train[1].max()+1
    bu = np.full(nusers, 0, dtype=float)
    bi = np.full(nusers, 0, dtype=float)
    P = np.random.normal(loc = 0, scale = 0.1, size=(nusers, k))
    Q = np.random.normal(loc = 0, scale = 0.1, size=(nitems, k))
    error = list()
    for l in range(0, miter):
        sq_error = 0
        for j in train.index:
            u = train[0][j]
            i = train[1][j]
            r_ui = train[2][j]
            pred = global_mean + bu[u] + bi[i] + np.dot(P[u, ], Q[i, ])
            e_ui = r_ui - pred
            sq_error += e_ui**2
            bu[u] += lr * e_ui # - reg * bu[u]
            bi[i] += lr * e_ui # - reg * bi[i]
            for f in range(k):
                temp_uf = P[u, f]
                P[u, f] = P[u, f] + lr * (e_ui * Q[i, f] - reg * P[u, f])
                Q[i, f] = Q[i, f] + lr * (e_ui * temp_uf - reg * Q[i, f])
        error.append(math.sqrt(sq_error/len(train)))

    return { "global_mean": global_mean, "bu": bu, "bi": bi, "P": P, "Q": Q, "error": error }

In [6]:
# svdopt = svdopt(train[:401838], 4)

In [7]:
# Predizendo nota
def predict(model, u, i):
    return model["global_mean"] + model["bu"][u] + model["bi"][i] + np.dot(model["P"][u], model["Q"][i])

In [8]:
# Avaliando predições
def rmse(model, test):
    sum_err = 0
    for t in test:
        u = t[0]
        i = t[1]
        r_ui = t[2]
        pred = predict(model, u, i)
        error = (r_ui - pred)**2
        sum_err += error
    return math.sqrt(sum_err/len(test))

In [9]:
def validation(model, data, train_split = 0.75, k = 5, lr = 0.05, reg = 0.02,miter = 1):
    data = data.sample(frac=1)
    train_len = int(train_split * len(data))
    train = data[:train_len]
    test = data[train_len:].values

    start_time = time.time()
    model = model(train, k, lr, reg, miter)
    print("Tempo de treinamento em segundos: ", time.time() - start_time)
    start_time = time.time()
    print("RMSE :", rmse(model, test))
    print("Tempo de predição em segundos: ", time.time() - start_time)

In [10]:
# rmse(svdopt, train.values[401838:])

In [11]:
validation(svdopt, train, 0.75, k=5, lr = 0.02, reg = 0.3, miter = 15)

Tempo de treinamento em segundos:  440.7574529647827
RMSE : 0.9104338997036565
Tempo de predição em segundos:  0.9981229305267334


## Execução 1 treino 75% k = 5
Tempo de treinamento em segundos:  34.38102984428406 <br />
RMSE : 0.9249984482014513 <br />
Tempo de predição em segundos:  1.005342960357666 <br />

## Execução 2 treino 75% k = 10
Tempo de treinamento em segundos:  39.39285969734192 <br />
RMSE : 0.9260717056726036 <br />
Tempo de predição em segundos:  1.144561767578125 <br />

## Execução 3 treino 75% k = 15
Tempo de treinamento em segundos:  45.229923248291016 <br />
RMSE : 0.923347732264856 <br />
Tempo de predição em segundos:  1.0131878852844238 <br />

## Execução 4 treino 75% k = 2
Tempo de treinamento em segundos:  28.49231481552124 <br />
RMSE : 0.9246241256750587 <br />
Tempo de predição em segundos:  1.006953477859497 <br />

## Execução 5 treino 75% k = 2
Tempo de treinamento em segundos:  34.321707010269165 <br />
RMSE : 0.9273759167030009 <br />
Tempo de predição em segundos:  1.3066084384918213\n <br />

## Execução 6 treino 75% k = 2
Tempo de treinamento em segundos:  40.08066439628601 <br />
RMSE : 0.9248382110380263 <br />
Tempo de predição em segundos:  1.3647074699401855 <br />

## Execução 7 treino 75% k = 5, lr = 0.05, reg = 0.2
Tempo de treinamento em segundos:  33.6286256313324 <br />
RMSE : 0.9231056768864376 <br />
Tempo de predição em segundos:  1.073127269744873 <br />

## Execução 9 treino 75% k = 5, lr = 0.2, reg = 0.2
Tempo de treinamento em segundos:  43.79306411743164 <br />
RMSE : 1.0033836401152396 <br />
Tempo de predição em segundos:  1.1530001163482666 <br />

## Execução 10 treino 75% k = 5, lr = 0.02, reg = 0.2
Tempo de treinamento em segundos:  32.61998724937439 <br />
RMSE : 0.9191592434431628 <br />
Tempo de predição em segundos:  0.8447728157043457 <br />

## Execução 11 treino 75% k = 5, lr = 0.02, reg = 0.3
Tempo de treinamento em segundos:  32.29331994056702 <br />
RMSE : 0.9187486544684991 <br />
Tempo de predição em segundos:  1.0716979503631592 <br />

## Execução 12 treino 75% k = 5, lr = 0.02, reg = 0.3
Tempo de treinamento em segundos:  24.829047679901123 <br />
RMSE : 0.9293885769817292 <br />
Tempo de predição em segundos:  1.0053107738494873 <br />

## Execução 13 treino 75% k = 5, lr = 0.02, reg = 0.3, miter = 15
Tempo de treinamento em segundos:  440.7574529647827 <br />
RMSE : 0.9104338997036565 <br />
Tempo de predição em segundos:  0.9981229305267334 <br />